In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고가 뜨지 않도록..
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 출력 창 청소
from IPython.display import clear_output

import time


# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 딥러닝
import tensorflow as tf

# 딥러닝 모델 구조를 정의하는 것
from tensorflow.keras.models import Sequential
# 층 모델(은닉층과 출력층)
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 것
from tensorflow.keras.layers import Activation

# GPU 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
# GPU가 있다면..
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)


In [2]:
# 랜덤 시드 설정
# 딥러닝 모델의 기중치와 바이어스는 랜덤값으로 시작한다
# 학습 모델 보정시 일정한 패턴을 랜덤으로 설정해야 정확한 테스트가 가능하기 때문에 랜덤시드를 설정하는 것이 좋다. 
np.random.seed(1)
tf.random.set_seed(1)


In [3]:
# 데이터를 읽어온다. 
df1 = pd.read_csv('data/ThoraricSurgery.csv', header=None)
df1


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,293,1,3.80,2.80,0,0,0,0,0,0,12,0,0,0,1,0,62,0
1,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60,0
2,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66,1
3,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80,1
4,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,98,6,3.04,2.40,2,0,0,0,1,0,11,0,0,0,1,0,76,0
466,369,6,3.88,2.72,1,0,0,0,1,0,12,0,0,0,1,0,77,0
467,406,6,5.36,3.96,1,0,0,0,1,0,12,0,0,0,0,0,62,0
468,25,8,4.32,3.20,0,0,0,0,0,0,11,0,0,0,0,0,58,1


In [4]:
# 불필요한 데이터 제거
df1.drop(0, axis=1, inplace=True)
df1


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,3.80,2.80,0,0,0,0,0,0,12,0,0,0,1,0,62,0
1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60,0
2,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66,1
3,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80,1
4,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,6,3.04,2.40,2,0,0,0,1,0,11,0,0,0,1,0,76,0
466,6,3.88,2.72,1,0,0,0,1,0,12,0,0,0,1,0,77,0
467,6,5.36,3.96,1,0,0,0,1,0,12,0,0,0,0,0,62,0
468,8,4.32,3.20,0,0,0,0,0,0,11,0,0,0,0,0,58,1


In [5]:
# 입력과 결과로 나눈다.
X = df1.drop(17, axis=1)
y = df1[17]

display(X)
display(y)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,3.80,2.80,0,0,0,0,0,0,12,0,0,0,1,0,62
1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60
2,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66
3,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80
4,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,6,3.04,2.40,2,0,0,0,1,0,11,0,0,0,1,0,76
466,6,3.88,2.72,1,0,0,0,1,0,12,0,0,0,1,0,77
467,6,5.36,3.96,1,0,0,0,1,0,12,0,0,0,0,0,62
468,8,4.32,3.20,0,0,0,0,0,0,11,0,0,0,0,0,58


0      0
1      0
2      1
3      1
4      0
      ..
465    0
466    0
467    0
468    1
469    0
Name: 17, Length: 470, dtype: int64

In [6]:
# 신경망 구조를 설계한다.
# 각 층의 구조를 관리하는 객체를 생성한다.
model = Sequential()

# 층을 쌓는 부분
# 케라스는 은닉층과 출력층만 설정한다
# 첫 번째 은닉층을 설정할 때 input_dim을 통해 입력층 노드의 개수를 설정한다. 
# 입력층 노드의 개수는 입력 데이터 컬럼의 수와 일치해야 한다. 
# 은닉층의 개수와 노드의 수에 정해진 정답은 없다. 
# 은닉층의 개수와 노드의 수가 많으면 학습 효율은 좋아지고 시간은 오래걸리며 과적합이 될 확률이 높아진다. 
model.add(Dense(30, input_dim=16))

# 활성화 함수
# relu : 역전파시 기울기 소실 문제를 해소할 수 있다.
model.add(Activation('relu'))

# 출력층
# 마지막에 구성하는 것은 무조건 출력층이다.
# 노드의 개수는 다음과 같은 규칙을 따른다.
# 회귀 : 1
# 2진 분류 : 1
# 다중 분류 : 결과 값의 종류 수
model.add(Dense(1))

# 활성화함수
# 회귀 : 설정하지 않는다.
# 2진 분류 : sigmoid
# 다중 분류 : softmax
model.add(Activation('sigmoid'))


In [7]:
# 위에서 설정한 구조를 통해 신경망을 생성한다.
# loss : 손실함수. 오차정도를 계산하는 함수
# metrics : 평가함수. 실제값과 예측값을 비교 평가하는 함수
# optimizer : 경사하강법
model.compile(loss='mean_squared_error', optimizer='adam', 
              metrics=['accuracy'])


In [9]:
# 학습한다.
# 첫번째, 두번째 : 입력데이터, 결과데이터
# epochs : 학습 횟수
# batch_size : 한 번에 몇 개의 로우를 메모리에 올릴 것인지 설정.
# 이 값이 크면 한번에 메모리에 올라가는 양이 많기 때문에 속도가 빠르다. 하지만 메모리가 부족할 정도로 설정해주면 오류가 발생한다.
# cpu와 gpu 운영의 차이는 5 ~ 20배 가량의 시간 차이가 발생한다.
# 공부단계에는 gpu 를 사용하겠다면 batch_size를 적게 잡아주는게 좋다.
# cpu를 사용하겠다면 batch_size를 확 늘려주는게 좋다.
model.fit(X, y, epochs=100, batch_size=10)


Epoch 1/100
47/47 [==============================] - 2s 3ms/step - loss: 0.1939 - accuracy: 0.7596
Epoch 2/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1425 - accuracy: 0.8511
Epoch 3/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1318 - accuracy: 0.8511
Epoch 4/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1330 - accuracy: 0.8489
Epoch 5/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1277 - accuracy: 0.8511
Epoch 6/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1267 - accuracy: 0.8511
Epoch 7/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1280 - accuracy: 0.8511
Epoch 8/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1295 - accuracy: 0.8511
Epoch 9/100
47/47 [==============================] - 0s 3ms/step - loss: 0.1243 - accuracy: 0.8511
Epoch 10/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1249 - accuracy: 0.8511
Epoch 11/

47/47 [==============================] - 0s 2ms/step - loss: 0.1204 - accuracy: 0.8532
Epoch 83/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1233 - accuracy: 0.8489
Epoch 84/100
47/47 [==============================] - 0s 3ms/step - loss: 0.1198 - accuracy: 0.8511
Epoch 85/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1210 - accuracy: 0.8468
Epoch 86/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1187 - accuracy: 0.8511
Epoch 87/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1207 - accuracy: 0.8489
Epoch 88/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1183 - accuracy: 0.8553
Epoch 89/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1195 - accuracy: 0.8511
Epoch 90/100
47/47 [==============================] - 0s 3ms/step - loss: 0.1227 - accuracy: 0.8532
Epoch 91/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1186 - accuracy: 0.8489
Epoch 92/100


In [11]:
# 평가
r1 = model.evaluate(X,y)

print(f'손실률 : {r1[0]}')
print(f'정확도 : {r1[1]}')
# 손실률이 낮은게 좋음

15/15 [==============================] - 0s 4ms/step - loss: 0.1196 - accuracy: 0.8511
손실률 : 0.11959640681743622
정확도 : 0.8510638475418091


In [12]:
# 학습 데이터를 통한 예측
result = (model.predict(X) > 0.5).astype('int')
result


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    